<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto (Canada)</font></h1>

In [1]:
import pandas as pd  # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np  # library to handle data in a vectorized manner

!conda install -c conda-forge wikipedia --yes
import wikipedia as wp

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Create a dataframe with Toronto data

First import the libraries that will be needed.

Get Toronto data from Wikipedia page.

In [2]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df = df[df['Borough'] != 'Not assigned']   # Ignore cells with a borough that is Not assigned.
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combine neighbourhoods in the same Postal Code Areas into one row with the neighborhoods separated with a comma.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Use the .shape method to print the number of rows of your dataframe.

In [3]:
columnnames = df.columns
df_new = pd.DataFrame(columns=columnnames)
Postcode = df['Postcode'].unique()

for pc in Postcode:
    pos = df['Postcode'] == pc
    borough = df['Borough'][pos].unique()
    neighbourhoods = df['Neighbourhood'][pos]
    neighbourhoods.mask(neighbourhoods=='Not assigned',borough, inplace = True)
    neighbourhoods = neighbourhoods.str.cat(sep=',')
    df_new = df_new.append({'Postcode':pc, 'Borough':np.array2string(borough), 'Neighbourhood':neighbourhoods}, ignore_index=True)
    
print('The dataframe has {} rows.'.format(df_new.shape[0]))
df_new.head()

The dataframe has 103 rows.


,Postcode,Borough,Neighbourhood
0,M3A,['North York'],Parkwoods
1,M4A,['North York'],Victoria Village
2,M5A,['Downtown Toronto'],Harbourfront
3,M6A,['North York'],"Lawrence Heights,Lawrence Manor"
4,M7A,['Downtown Toronto'],Queen's Park


## 2. Get the latitude and the longitude coordinates of each neighborhood

Install and imprt Geocoder library

In [4]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [5]:
# initialize variables
Latitude=[]
Longitude=[]

for pc in Postcode:
    # loop until you get the coordinates
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])


In [6]:
# Add the latitude and longitude data of each postal code area to the dataframe
df_new['Latitude'] = Latitude
df_new['Longitude'] = Longitude

df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,['North York'],Parkwoods,43.752420,-79.329242
1,M4A,['North York'],Victoria Village,43.730600,-79.313265
2,M5A,['Downtown Toronto'],Harbourfront,43.650295,-79.359166
3,M6A,['North York'],"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,M7A,['Downtown Toronto'],Queen's Park,43.661150,-79.391715


## 3. Explore and cluster the neighborhoods in Toronto

#### Create a map of Toronto with neighborhoods superimposed on top.

In [7]:
# Install and import Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [8]:
# Get latitude and longitude coordinates of Toronto
g = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in boroughs that contain the word Toronto.

First, from the dataframe select the rows corresponding to a borough containing the word Toronto.

In [9]:
pos = df_new['Borough'].str.contains('Toronto')
toronto_data = df_new[pos]
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,['Downtown Toronto'],Harbourfront,43.650295,-79.359166
4,M7A,['Downtown Toronto'],Queen's Park,43.661150,-79.391715
9,M5B,['Downtown Toronto'],"Ryerson,Garden District",43.657363,-79.378180
15,M5C,['Downtown Toronto'],St. James Town,43.651210,-79.375481
19,M4E,['East Toronto'],The Beaches,43.676531,-79.295425


Visualize Toronto closer and the neighborhoods in it.

In [10]:
# create map of closer Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = '5TQ1R1DIHISONMOHVOCIIBKYC2UCGXLRW0JD5G30JOEYFMI0' # your Foursquare ID
CLIENT_SECRET = 'QY1QBUEMUO0KWTVCUC0OZQ2BBYQ32M00RDY2HZPWWBIDCIUJ' # your Foursquare Secret
VERSION = '20200124' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5TQ1R1DIHISONMOHVOCIIBKYC2UCGXLRW0JD5G30JOEYFMI0
CLIENT_SECRET:QY1QBUEMUO0KWTVCUC0OZQ2BBYQ32M00RDY2HZPWWBIDCIUJ


#### Explore neighborhoods in central Toronto

Get the first 100 venues within a radius of 500m for each entry in the dataframe.

In [12]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
1,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
2,Harbourfront,43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
3,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
4,Harbourfront,43.650295,-79.359166,Cacao 70,43.650067,-79.360723,Dessert Shop


Check how many venues were returned for each neighborhood

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,63,63,63,63,63,63
"Brockton,Exhibition Place,Parkdale Village",67,67,67,67,67,67
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",69,69,69,69,69,69
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",76,76,76,76,76,76
Christie,11,11,11,11,11,11


Find out how many unique categories can be curated from all the returned venues

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


#### Analyze each neighborhood

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.020000,0.000000,0.010000,0.030000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.00000,

Print each neighborhood along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4          Bar  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.05
2    Restaurant  0.03
3   Cheese Shop  0.03
4      Beer Bar  0.03


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.06
2  Furniture / Home Store  0.06
3              Restaurant  0.06
4          Sandwich Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.04
2          Bar  0.04
3   Steakhouse  0.04
4         Café  0.03


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2                Café  0.04
3                 Bar  0.04
4      

Put that into a *pandas* dataframe

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gym,Breakfast Spot,Restaurant,Japanese Restaurant,Gastropub,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Hotel,Seafood Restaurant,Breakfast Spot,Café,Cheese Shop,Restaurant,Beer Bar,Farmers Market
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Restaurant,Café,Furniture / Home Store,Bakery,Sandwich Place,Spa,Bar,Juice Bar,Hotel
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Hotel,Steakhouse,Bar,Gym,Café,Pub,Italian Restaurant,Sushi Restaurant,Seafood Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Café,Bar,Spa,Speakeasy,Electronics Store,Restaurant,Bakery,Park


#### Cluster the neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [97]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [98]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,['Downtown Toronto'],Harbourfront,43.650295,-79.359166,0.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Ice Cream Shop,Hotel,Shoe Store,Café,Gastropub,Gym Pool
4,M7A,['Downtown Toronto'],Queen's Park,43.661150,-79.391715,0.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Gym,Salad Place,Juice Bar,Bookstore,Fried Chicken Joint,Bubble Tea Shop
9,M5B,['Downtown Toronto'],"Ryerson,Garden District",43.657363,-79.378180,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Japanese Restaurant,Tea Room,Restaurant,Bookstore
15,M5C,['Downtown Toronto'],St. James Town,43.651210,-79.375481,0.0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Bakery,Cosmetics Shop,Beer Bar,Breakfast Spot,Hotel
19,M4E,['East Toronto'],The Beaches,43.676531,-79.295425,0.0,Health Food Store,Pub,Trail,Women's Store,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


Visualize the resulting clusters

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,['Downtown Toronto'],0.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Ice Cream Shop,Hotel,Shoe Store,Café,Gastropub,Gym Pool
4,['Downtown Toronto'],0.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Gym,Salad Place,Juice Bar,Bookstore,Fried Chicken Joint,Bubble Tea Shop
9,['Downtown Toronto'],0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Japanese Restaurant,Tea Room,Restaurant,Bookstore
15,['Downtown Toronto'],0.0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Bakery,Cosmetics Shop,Beer Bar,Breakfast Spot,Hotel
19,['East Toronto'],0.0,Health Food Store,Pub,Trail,Women's Store,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
20,['Downtown Toronto'],0.0,Coffee Shop,Cocktail Bar,Hotel,Seafood Restaurant,Breakfast Spot,Café,Cheese Shop,Restaurant,Beer Bar,Farmers Market
24,['Downtown Toronto'],0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Bakery,Plaza,Tea Room,Sandwich Place,Bookstore,Bubble Tea Shop,Restaurant
25,['Downtown Toronto'],0.0,Café,Grocery Store,Playground,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store,Coffee Shop,Fish Market,Fish & Chips Shop
30,['Downtown Toronto'],0.0,Coffee Shop,Café,Hotel,Steakhouse,Gym,Breakfast Spot,Restaurant,Japanese Restaurant,Gastropub,Bar
31,['West Toronto'],0.0,Park,Furniture / Home Store,Pet Store,Bakery,Café,Middle Eastern Restaurant,Bus Line,Pharmacy,Brazilian Restaurant,Bar


Cluster 1 has a lot of  entertainment venues nearby like coffee shops, restaurants and theaters, so it is a good neighbourhood for doing fun activities.

Cluster 2

In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,['Central Toronto'],1.0,Bus Line,Swim School,Lawyer,Women's Store,Event Space,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


Cluster two has more venues for daily activities like transportation (bus line), shool and shopping.

Cluster 3

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,['Central Toronto'],2.0,Playground,Gym Pool,Park,Garden,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
83,['Central Toronto'],2.0,Playground,Gym,Trail,Park,Women's Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
91,['Downtown Toronto'],2.0,Playground,Candy Store,Grocery Store,Park,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


Cluster 3 is good for raising kids since the most common venue is playgrounds, and there are also parks and trails for day trips. In addition, there are shops to get everyday necessities.

Cluster 4

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,['Central Toronto'],3.0,Home Service,Park,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


Cluster 4 has a wide variety of restaurants.

Cluster 5

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,['East Toronto'],4.0,Bus Line,Discount Store,Grocery Store,Park,Women's Store,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market


Cluster 5 is also good for dialy life because of the abundancy of stores and also for being well connected by bus.